In [2]:
import pandas as pd  
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from time import time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import MultinomialNB
import pickle

In [2]:
df = pd.read_csv('preproccesedtweet.csv')


In [3]:
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1596041 entries, 0 to 1596040
Data columns (total 3 columns):
Unnamed: 0    1596041 non-null int64
text          1596041 non-null object
target        1596041 non-null int64
dtypes: int64(2), object(1)
memory usage: 36.5+ MB


In [4]:
X = df.text
y = df.target

In [2]:
vectorizer = TfidfVectorizer(max_features=80000,ngram_range=(1, 3))
multi_NB = MultinomialNB()

In [6]:
pipeline = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', multi_NB)
])

In [7]:

X_train, X_validation_and_test, y_train, y_validation_and_test = train_test_split(X, y, test_size=.05, random_state=42)
X_validation, X_test, y_validation, y_test = train_test_split(X_validation_and_test, y_validation_and_test, test_size=.5, random_state=42)

In [8]:
if len(X_test[y_test == 0]) / (len(X_test)*1.) > 0.5:
    null_accuracy = len(X_test[y_test == 0]) / (len(X_test)*1.)
else:
    null_accuracy = 1. - (len(X_test[y_test == 0]) / (len(X_test)*1.))

In [9]:
fitted_model = pipeline.fit(X_train, y_train)

In [1]:
lu.drop(lu[lu['number_of_tweet'] < 500].index, inplace = True) 

NameError: name 'lu' is not defined

In [ ]:
f1_score()

In [10]:
pred = fitted_model.predict(X_test)
score = accuracy_score(y_test, pred)
print("The null accuracy is " + str(null_accuracy))
print("The multinomial bayes classifier accuracy is " + str(score))

The null accuracy is 0.5015538068267255
The multinomial bayes classifier accuracy is 0.8019648137937948


In [11]:
pickle.dump(fitted_model, open('multiNB_model.sav', 'wb'))



In [3]:
loaded_model = pickle.load(open('multiNB_model.sav', 'rb'))


In [11]:
pred1 = fitted_model.predict(X_test)
score = accuracy_score(y_test, pred1)
print(score)

0.8019648137937948


In [6]:
clean1 = tweet_cleaner_updated('@LUca5pHam I HaTe ApPlE StOcK')
clean2 = tweet_cleaner_updated('@BigBoseBoulder I only buy google pixels cause they are the best lol rofl')

In [7]:
print(clean1)
print(clean2)

hate apple stock
only buy google pixels cause they are the best lol rofl


In [8]:
loaded_model.predict([clean1, clean2])

array([0, 1])

In [5]:
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

pat1 = r'@[A-Za-z0-9_]+'
pat2 = r'https?://[^ ]+'
combined_pat = r'|'.join((pat1, pat2))
www_pat = r'www.[^ ]+'
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

def tweet_cleaner_updated(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom_removed = souped
    stripped = re.sub(combined_pat, '', bom_removed)
    stripped = re.sub(www_pat, '', stripped)
    lower_case = stripped.lower()
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled)
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1]
    return (" ".join(words)).strip()